In [2]:
#place your bprna params
#see https://www.kaggle.com/code/its7171/how-to-generate-augmentation-data for more details
import os
perl_lib_path = '/home/hoyso/perl5/lib/perl5'
os.environ['PERL5LIB'] = perl_lib_path

In [3]:
import pandas as pd
from tqdm.autonotebook import tqdm

/tmp/ipykernel_836981/158874326.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
def get_predicted_loop_type(sequence, structure, debug=False, pid=0):
    with open(f'a{pid}.dbn', 'w') as file:
        file.write(sequence + '\n')
        file.write(structure + '\n')

    os.system(f'perl bpRNA/bpRNA.pl a{pid}.dbn')

    result = [l.strip('\n') for l in open(f'a{pid}.st')]
    if debug:
        print(sequence)
        print(structure)
        print(result[5])

    return result[5]

In [5]:
import shutil
from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
import threading

def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def get_predicted_loop_type_chunk(chunk):
    seq = chunk.sequence
    mfe = chunk.mfe_eternafold
    #mfe = chunk.mfe_contrafold
    thread_id = threading.get_ident()
    result = []
    for s,m in tqdm(zip(seq,mfe)):
        result.append(get_predicted_loop_type(s,m,False,thread_id))
    os.remove(f'a{pid}.dbn')
    os.remove(f'a{pid}.st')
    return result

In [6]:
train_df = pd.read_parquet('train_data_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(get_predicted_loop_type_chunk)(x)
    for i,x in enumerate(split_dataframe(train_df, 10000))
)
train_df['looptype_eternafold'] = np.concatenate(result, axis=0)
train_df.to_parquet('train_data_new.parquet')
del train_df

AttributeError: 'DataFrame' object has no attribute 'mfe_eternafold'

In [ ]:
test_df = pd.read_parquet('test_sequences_new.parquet')

result = Parallel(n_jobs=cpu_count())(
    delayed(get_predicted_loop_type_chunk)(x)
    for i,x in enumerate(split_dataframe(test_df, 10000))
)
test_df['looptype_eternafold'] = np.concatenate(result, axis=0)
test_df.to_parquet('test_sequences_new.parquet')
del test_df